# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откройте файл с данными и изучите общую информацию

Импортируем библиотеку Pandas

In [1]:
import pandas as pd 

#Импортируем лемматизатор
from pymystem3 import Mystem

m = Mystem()

Прочитаем файл data, сохраним его в переменную df, и сразу выведем первые 5 строк.

In [2]:
df=pd.read_csv('/datasets/data.csv')
df.head(40)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим общие даныне по таблице

In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Проверяем пустые значения

In [4]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверяем наличие дубликатов в данных

In [6]:
df.duplicated().sum()

54

Проверим есть ли дубликаты в различных категориях по регистру в образовании, семейном положении, виде деятельности, и назначении кредита.

In [7]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [8]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [9]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [10]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**


Обнаружены следующие недостатки в данных:

1) Есть пропуски в days_employed, total_income (необходимо понять вид пропуска, причину появления и устранить их)

2) Общий трудовой стаж в днях выражен в float еще и с отрицательными значениями. (Необходимо поправить и сменить тип данных), так же есть аномальные значения, например количество рабочих дней превышает 300 000 дней, столько люди не живут. Проверим эти данные.

3) В данных есть 54 дубликата, а так же большое количество дубликатов по регистру в категории образования.

4) Среди назначений кредитов огромное количество одинковых целей, но записанных другой фразой. Необходима лемматизация.

## Предобработка данных

### Обработка пропусков

Пропущенные данные в категориях: "общий трудовой стаж" и "ежемесячный доход". Такие данные являются количественными переменными.

Давайте создадим датасет с этими значениями и попробуем найти что-то интересное:

In [11]:
null_data = df[(df.isnull().any(axis=1)) & (df.isna().any(axis=1))]
null_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Проверим какие категории людей не указали данные.

In [12]:
null_data['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

#### Обработка пропусков - стаж

**СОТРУДНИК. КОМПАНЬОН. ГОССЛУЖАЩИЙ.**

Люди из категорий: "сотрудник", "компаньон", "госслужащий" - очень схожие по смыслу. Одни являются наемными сотрудниками трудящимися в коммерческих организациях, другие являются собственниками бизнеса, третьи являются по сути наемными сотрудниками, но трудятся на государство. Пропуски могут быть обусловлены сложностью вычисления стажа, так как люди могли часто менять работу с перерывами. Некоторые могли работать неофициально или на договорной основе, как самозанятые, соответственно официального стажа нет, либо по каким-то причинам нет возможности официалньо поддтвердить стаж.

Вероятно наилучшим способом вычисления стажа для данных категорий будет использование функции принимающей на вход возраст. Использование среднего арифметического или медианы будет не лучшим решением, так как возраст может повлиять на величину. 

Напишем функцию для вычисления стажа: 

In [13]:
def qty_days_empl(year):
    '''
    Функция вычисляет количество рабочих дней у объекта c помомщи возраста и количества рабочих дней в году
    Условно считается, что объект начал набирать рабочий стаж с 18ти лет
    Количество рабочих дней в году 247

    ======================================

    Аргументы:
                    year (Int): Возраст объекта

    Возвращает:
                    qty_days_empl (Int): Рабочий стаж в днях

    '''
    qty_of_working_year = year-18
    qty_days_empl = qty_of_working_year*247
    return qty_days_empl


print('Строка документации: \n', qty_days_empl.__doc__)

Строка документации: 
 
    Функция вычисляет количество рабочих дней у объекта c помомщи возраста и количества рабочих дней в году
    Условно считается, что объект начал набирать рабочий стаж с 18ти лет
    Количество рабочих дней в году 247


    Аргументы:
                    year (Int): Возраст объекта

    Возвращает:
                    qty_days_empl (Int): Рабочий стаж в днях

    


Проверим функцию:

In [14]:
qty_days_empl(19)

247

Заполним пропуски стажа для категории "сотрудник" при помощи вышенаписанной функции.

In [15]:
df.loc[(df['income_type']=='сотрудник'), 'days_employed'] = df['days_employed'].fillna(qty_days_empl(df['dob_years']))

In [16]:
#Проверяем количество пропусков должно уменьшиться
df['days_employed'].isnull().sum()

1069

Заполним пропуски стажа для категории "госслужащий" при помощи вышенаписанной функции.

In [17]:
df.loc[(df['income_type']=='госслужащий'), 'days_employed'] = df['days_employed'].fillna(qty_days_empl(df['dob_years']))

In [18]:
#Проверяем количество пропусков должно уменьшиться
df['days_employed'].isnull().sum()

922

Заполним пропуски стажа для категории "компаньон" при помощи вышенаписанной функции.

In [19]:
df.loc[(df['income_type']=='компаньон'), 'days_employed'] = df['days_employed'].fillna(qty_days_empl(df['dob_years']))

In [20]:
#Проверяем количество пропусков должно уменьшиться
df['days_employed'].isnull().sum()

414

In [21]:
#Проверяем адекватность заполнения.
df.head(40)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Функция зполнила все необходимые пропуски, пенсионеры и предприниматели остались нетронутыми.

**ПЕНСИОНЕРЫ**

Люди из данной категории могли не укзать стаж ввиду того, что эта информация может быть избыточной так как человек на пенсии и его стаж не будет играть ключевой роли в решении о выдаче заемных средств. Так же кто-то из пенсионеров мог просто не укзаать данные в виду трудности вычисления стажа. Так же можно заметить в даннхы "молодых пенсионеров", скорее всего это люди которые получают пенсию в виду жизненных трудностей. По сути, они указали ежемесячный доход, а банк будет понимать, что у данной группы людей есть стабильная часть в виде пенсии по инвалидности.

Мы можем пойти двумя путями: 

1) Дропнуть пропущенные значения.

2) Взять среднее арифметическое значение. Так как старт пенсионного возраста известен, а вероятность аномальных выбросов будет минимальна. 

Мы воспользуемся вторым методом, Всего по пенсионерам 3856 значений, пропущенных 413. Количество пропущенных значений - 10,7% от всей суммы. Такое количество выброшенных значений может повлиять на выводы и модель в будущем.

Познакомимся с данными

In [22]:
#Проверим фрейм и данные
df[df['income_type']=='пенсионер'].info()
df[df['income_type']=='пенсионер'].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3856 entries, 4 to 21521
Data columns (total 12 columns):
children            3856 non-null int64
days_employed       3443 non-null float64
dob_years           3856 non-null int64
education           3856 non-null object
education_id        3856 non-null int64
family_status       3856 non-null object
family_status_id    3856 non-null int64
gender              3856 non-null object
income_type         3856 non-null object
debt                3856 non-null int64
total_income        3443 non-null float64
purpose             3856 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 391.6+ KB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3856.000000,3443.000000,3856.000000,3856.000000,3856.000000,3856.000000,3443.000000
mean,0.132002,365003.491245,59.063019,0.913900,0.985737,0.056017,137127.465690
std,1.014271,21069.606065,7.618526,0.510185,1.315202,0.229984,80246.953231
min,-1.000000,328728.720605,0.000000,0.000000,0.000000,0.000000,20667.263793
25%,0.000000,346649.346146,56.000000,1.000000,0.000000,0.000000,82881.443465
50%,0.000000,365213.306266,60.000000,1.000000,0.000000,0.000000,118514.486412
75%,0.000000,383231.396871,64.000000,1.000000,2.000000,0.000000,169700.433010
max,20.000000,401755.400475,74.000000,4.000000,4.000000,1.000000,735103.270167


Средние и медианные значения стажа аномально высокие, выкинуть из данных не получится, так как все указавшие стаж пенсионеры имеют аномальные значения. Буду использовать данные как есть, но напишу в подразделение предоставившее данные, возможно какая-то ошибка в вычислении или же  возраст и размер пенсии в данном случае более весомые факторы, по этой причине система могла выдать или рассчитать такие аномальные значения как 300 000 и более. Так же возможно данные указаны не в днях, а каких-то других единицах измерения.

Интересность вызвает столбец "children" где есть одно отрицательно значение, а так же одно больше значение в 20, данное значение сильно выше среднего.

In [23]:
df.loc[(df['income_type']=='пенсионер'), 'days_employed']=df['days_employed'].fillna(df.loc[(df['income_type']=='пенсионер'), 'days_employed'].mean())

Проверяем данные должен остаться один предприниматель.

In [24]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [25]:
df['days_employed'].isnull().sum()

1

**Вывод**

Мы заменили пропуски пенсионеров средним значением. У нас остался необработанным лишь один предприниматель. Заполним его следующим знаением.

**Предприниматели**

Исходя из данных выше, мы видим, что в данных всего 2 предпринимателя, и один из них не указал данные о своем стаже. есть два пути: 

1) указать такое же значение для предпринимателя с пустым значением. Но считаю такой метод не очень удачным, так как совсем не факт, что второй предприниматель имеет такой же стаж. 

2) Можно выкинуть из данных данное пустое значение. Так как на фоне 21000 строк потеря одной строки не исказит данных.

Выбираю второй вариант

In [26]:
#узнаем индекс ненужной строки.
df[df['income_type']=='предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


In [27]:
#Дропаем пустое значение
df = df.drop(index=[5936])
df = df.reset_index(drop=True)

In [28]:
#Проверяем данные
df[df['income_type']=='предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
18696,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


**Вывод**

У нас остался единственный предприниматель. "Удаленный" предприниматель так же имел пропуск в "заработной плате".

#### Обработка пропусков - зароботная плата

Приступаем к заполнению пропущенных значений "total_income". Чтобы качественно заполнить данный столбец, нам необходимо категоризировать людей по возрастным группам, так как от сатаж может меняться заработная плата. Заполнять будем медианным значением, так как заработные платы от должности и предприятия разнятся и могут быть серьзеные выбросы. Так же необходимо учитывать образование и тип занятости людей.

Для снятия дубликатов в столбце "education" используем метод str.lower() - Приводим строки в "education" к нижнему регистру

In [29]:
df['education']=df['education'].str.lower()
#Сразу проверим данные
df['education'].value_counts()

среднее                15233
высшее                  5259
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Отлично, вместо 15ти категорий в "образовании" у нас осталось 5. Данный шаг мы выполнили раньше работ чистки дубликатов, так как данный шаг необходим чтобы качественно заполнить "total_income" медианными значениями.

Напишем функцию для категоризации возрастных групп. Для разделения людей на группы воспользуемся возратсной периодизацией психосоциального развития Эрика Эриксона, где люди делятся на следующие категории:

1) 18-39 лет - Молодость

2) 39-60 лет - Зрелость

3) 60 и больше - Старость

In [30]:
#Функция на вход принимает значение возраста и через блок услвоия каждому возрасту присваивает категорию и возвращает её.
def age_group_category(row):
    '''
    Функция присваивает категорию на основе значения столбца 'dob_years'.
    Возможны 3 категории:

    молодежь
    cтарики
    зрелые
    
    ======================================

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма.

    Возвращает:
                    string: заполняет ячейку категорией.

    '''
    years=row['dob_years']
    if years <= 39:
        return 'молодежь'
    elif years > 60:
        return 'cтарики'
    else:
        return 'зрелые'
    
print('Строка документации: \n', age_group_category.__doc__)

Строка документации: 
 
    Функция присваивает категорию на основе значения столбца 'dob_years'.
    Возможны 3 категории:

    молодежь
    cтарики
    зрелые
    

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма.

    Возвращает:
                    string: заполняет ячейку категорией.

    


Создадим столбец "age_group" и заполним её при помощи нашей функции:

In [31]:
df['age_group']=df.apply(age_group_category, axis=1)

Теперь заполним столбец столбец "total_income", воспользуемся функцией transform.

In [32]:
df['total_income'] = df['total_income'].fillna(df.groupby(['income_type','age_group', 'education'])['total_income'].transform('median'))

In [33]:
#Заполненный фрейм назовем df1
df1=df

Проверим наличие пропущенных значений

In [36]:
df1.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        1
purpose             0
age_group           0
dtype: int64

У нас остался одна незаполненная строка. Рассмотрим её.

In [37]:
null_data_1 = df1[(df.isnull().any(axis=1)) & (df.isna().any(axis=1))]
null_data_1

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
8141,0,11362.0,64,начальное,3,гражданский брак,1,F,госслужащий,0,NaN,сыграть свадьбу,cтарики


Вероятнее всего человек с таким набором групп единственный и он не указал доход. Мы можем убрать его из нашего фрейма, так как на фоне всего объема данных потеря этой строки не повлияет на результат.

In [38]:
#Дропаем пустое значение
df1 = df1.drop(index=[8141])
df1 = df1.reset_index(drop=True)

Вновь проверяем данные:

In [39]:
df1.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

**Вывод:**

Мы заполнили все пропущенные данные используя расзличные методы:

**Для заполнения стажа:**

-Для категории "пенсионер": мы использовали среднее арифметическое.

-Для категорий "сотрудник", "компаньон", "госслужащий": мы использовали написанную функцию для рассчета стажа исходя из возраста человека.

-Для категории "предприниматель": мы брали пропущенную строку, так как её количество в объеме всей выборки незначителньо, хотя если бы стояла задача как-то оценивать именно предпринимателей - такой подход бы не сработад. Тогда бы заполнял повторяющимся значением.

**Для заполнения доходов:**

Мы использовали вычисление медианных значений для каждой категории людей, с учетом возраста,  образования, и типом занятости.  Заполнили пропуски данными значениями. Использовали медиану в связи с разнящимися зарплатами на различных должностях и на различных предприятиях.

### Замена типа данных

Я увидел 2 типа данных которые стоит сменить:

    1) days_employed - нам волне достаточно целочисленных значений дня. Дроби-излишняя точность. 
Плюсом в столбце есть отрицательные значения.
    
    2) total_income - схожая ситуация. Дроби кардинально не влияют на уровень дохода. "Вес" символов после запятой не несет никакой нагрузки.


Сперва и исправим отрицателньые значения. Отрицателньые значения могли появится в результате автоматического распознования анкет. Люди могли просто ставить дефис. Лучше уточнить данную информацию у поздразделения откдуа мы получили данные, а пока  избавимся от отрицателньых значений. Используем для этого модуль числа. Заодно избавимся от отрицательного значения в графе "children"

In [40]:
#Избавляемся от "-" в столбце стажа
df1['days_employed']=df1['days_employed'].abs()
#Избавляемся от "-" в столбце детей
df1['children']=df1['children'].abs()
#Проверяем как все прошло.
df1.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21523.000000,21523.000000,21523.000000,21523.000000,21523.000000,21523.000000,2.152300e+04
mean,0.543326,67615.681037,43.291734,0.817172,0.972587,0.080890,1.653983e+05
std,1.379930,139256.131565,12.573976,0.547933,1.420374,0.272673,9.815328e+04
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1018.985844,33.000000,1.000000,0.000000,0.000000,1.077778e+05
50%,0.000000,2513.265737,42.000000,1.000000,0.000000,0.000000,1.435088e+05
75%,1.000000,6681.881103,53.000000,1.000000,1.000000,0.000000,1.968652e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Теперь заменим все вещественные типы данных на целочисленные.

In [41]:
df1['days_employed']=df1['days_employed'].astype('int')
df1['total_income']=df1['total_income'].astype('int')                                              

Так же для последующей лемматизации нам необходимо "purpose" перевести в string. Так как лемматизатор работает со строками 

In [42]:
df1['purpose']=df1['purpose'].astype('str')

#Проверим все данные
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21523 entries, 0 to 21522
Data columns (total 13 columns):
children            21523 non-null int64
days_employed       21523 non-null int64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null int64
total_income        21523 non-null int64
purpose             21523 non-null object
age_group           21523 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


**Вывод**

Мы заменили все отрицательные значения в столбцах на абсолютные, так же заменили вещественный типа данных на целочисленный. Теперь можем использовать вполне удобную и читабельную таблицу в нашем дальнейшем анализе.

### Обработка дубликатов

Ранее мы обнаружили следующие типы дубликатов:

1) задваивание - 54 дубликата в таблице

2) дубликаты по регистру - в столбце "education". 

Скорее всего во втором случае люди писали свое образование вручную, поэтому получили такую информацию. Можно предложить коллегам доработать анкеты выпадющим меню, чтобы клиент мог выбирать из предложенных категорий или в бумажной анкете сделать меню с галочками.

Сперва уберем задваивание, используем drop_duplicates().

In [43]:
df1=df1.drop_duplicates().reset_index(drop=True)
#Сразу проверим данные
df1.duplicated().sum()

0

**Вывод**

Отлично, мы выброслии 54 дубликата из данных. 

### Лемматизация

В назначении у нас перечиселны большое количество повторяющихся категорий, но записанных различными выражениями. Для построения удобной категоризации нам необходимо выделить леммы и желательно заменить или добавить новый столбец с категориями. Для лемматизиации будем использовать библиотеку PyMystem.

Ипортируем библиотеку и еще раз посмотрим категории:

In [44]:
#Еще раз посомтрим на данные 
df1['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           764
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                605
на покупку своего автомобиля              505
заняться высшим образованием      

По сути мы видим несколько категорий:

1) Недвижимость (ремонт, жилье, недвижимость)

2) Автомобиль

3) Образование

4) Мероприятия (свадьба)

Мы не  будем заменять данные в колонке "purpose", так как в будущем эти данные можно использовать для последующей более подробной категорезации, например недвижимость может быть жилая или коммерческая и тп.

Для решения нам необходим список всех значений, для того чтобы данный список передать в лемматизатор.

In [45]:
text=df1['purpose'].unique().tolist()
text

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [46]:
lemmas=[]

for string in text:
    lemmas += m.lemmatize(string)
print(lemmas)

['покупка', ' ', 'жилье', '\n', 'приобретение', ' ', 'автомобиль', '\n', 'дополнительный', ' ', 'образование', '\n', 'сыграть', ' ', 'свадьба', '\n', 'операция', ' ', 'с', ' ', 'жилье', '\n', 'образование', '\n', 'на', ' ', 'проведение', ' ', 'свадьба', '\n', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n', 'покупка', ' ', 'недвижимость', '\n', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'покупка', ' ', 'жилой', ' ', 'недвижимость', '\n', 'строительство', ' ', 'собственный', ' ', 'недвижимость', '\n', 'недвижимость', '\n', 'строительство', ' ', 'недвижимость', '\n', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n', 'строительство', ' ', 'жилой', ' ', 'недвижимость', '\n', 'жилье', '\n', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n', 'автомобиль', '\n', 'заниматься', ' ', 'образование', '\n', 'сделка', ' '

Мы получили леммы теперь посчитаем какие леммы являются самыми часто запрашиваемыми. Для этого импортируем счетчик Counter.
Заодно мы четко увидем леммы.

In [47]:
from collections import Counter

print(Counter(lemmas))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Отлично, леммы по которым мы можем  длеать категоризацю, а именно: 

1) недвижимость:недвижимость, жилье, ремонт
2) автомобиль: автомобиль 
3) образование: образование
4) мероприятие:свадьба

Отлично теперь нам необходимо создать столбец с категориями. Для заполнения данного столбца будем использовать функцию которая на вход будет принимать значение столбца "purpose", проводить лемматизацию и сравнивать леммы и в случае совпадения прописывать категорию.

In [48]:
def purpose_category(row):
    '''
    Функция присваивает категорию на основе значения столбца 'purpose'.
    Возможны 4 категории:

    мероприятие
    автомобиль
    образование
    недвижимость
    ======================================

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма.

    Возвращает:
                    string: заполняет ячейку категорией.

    '''
    lemmas = m.lemmatize(row['purpose'])
    if 'свадьба' in lemmas:
        return 'мероприятие'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'жилье' in lemmas or 'недвижимость' in lemmas or 'ремонт':
        return 'недвижимость'


print('Строка документации: \n', purpose_category.__doc__)

Строка документации: 
 
    Функция присваивает категорию на основе значения столбца 'purpose'.
    Возможны 4 категории:

    мероприятие
    автомобиль
    образование
    недвижимость

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма.

    Возвращает:
                    string: заполняет ячейку категорией.

    


Применяем функцию и создаем столбец с категориями

In [49]:
df1['category']=df1.apply(purpose_category, axis=1)
df1.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелые,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодежь,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодежь,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодежь,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,зрелые,мероприятие
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,молодежь,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,зрелые,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,зрелые,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,молодежь,мероприятие
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,зрелые,недвижимость


**Вывод**

При помощи лемматизации мы разбили 38 целей на 4 категории. Это упростит нашу работу в категоризации данных.

### Категоризация данных

Чтобы грамотно категоризировать данные нам необходимо ориентроваться на вопросы и задачи которые поставлены перед нами.

Перед нами стоят следующие вопросы:

**1) Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Нам необходимо разбить данные на 3 категории: 

 1) без детей 

 2) с детьми 

 3) многодетный родитель
                                              
**2) Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Будем пользоваться имеющейся категоризацией, так как от категорий может зависеть доход, например вдовцы и вдовы могут получать компенсацию, а разведенные люди могут оплачивать аллименты бывшим супругам если есть дети. Данные факторы могут влиять на доход и вылпату соответственно.

**3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?** 

Разобьем ровень дохода на 2 категории: 

 1) Выше среднего
 
 2) Ниже среднего
                                        
**4) Как разные цели кредита влияют на его возврат в срок?** 

В лемматизации мы уже назначили категории и будем использовать их для дальнейшего анализа.

Напишем функцию при помощи которой будем заполнять столбец категории родителей.

In [50]:
def parents_category(row):
    '''
    Функция присваивает категорию на основе значения столбца 'children'.
    Возможны 3 категории:

    без детей
    многодетный родитель
    с детьми

    ======================================

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    string: заполняет ячейку категорией.
    '''
    qty_of_chld = row['children']
    if qty_of_chld == 0:
        return 'без детей'
    elif qty_of_chld >= 3:
        return 'многодетный родитель'
    else:
        return 'с детьми'


print('Строка документации: \n', parents_category.__doc__)

Строка документации: 
 
    Функция присваивает категорию на основе значения столбца 'children'.
    Возможны 3 категории:

    без детей
    многодетный родитель
    с детьми


    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    string: заполняет ячейку категорией.
    


Применим нашу функцию к фрейму и создадим столбец "par_cat", где будут размещены категории людей.

In [51]:
df1['par_cat']=df1.apply(parents_category, axis=1)
df1.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,category,par_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелые,недвижимость,с детьми
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодежь,автомобиль,с детьми
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодежь,недвижимость,без детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодежь,образование,многодетный родитель
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,зрелые,мероприятие,без детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,молодежь,недвижимость,без детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,зрелые,недвижимость,без детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,зрелые,образование,без детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,молодежь,мероприятие,с детьми
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,зрелые,недвижимость,без детей


Напишем функцию при помощи которой будем назанччать категорию зарплат выше/ ниже средней, 
причем за среднюю зарплату мы будем считать мединное значение всех зарплат.

In [52]:
def income_category(row):
    '''
    Функция присваивает категорию на основе значения столбца 'total_income'.
    Возможны 2 категории:

    доход выше среднего
    доход ниже среднего

    ======================================

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    string: заполняет ячейку категорией.
    '''

    profit = row['total_income']
    if profit > df1['total_income'].median():
        return 'доход выше среднего'
    else:
        return 'доход ниже среднего'


print('Строка документации: \n', income_category.__doc__)

Строка документации: 
 
    Функция присваивает категорию на основе значения столбца 'total_income'.
    Возможны 2 категории:

    доход выше среднего
    доход ниже среднего


    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    string: заполняет ячейку категорией.
    


Применим нашу функцию к фрейму и создадим столбец "income_cat", где будут размещены категории людей по доходу.

In [53]:
df1['income_cat']=df1.apply(income_category, axis=1)
df1.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,category,par_cat,income_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,зрелые,недвижимость,с детьми,доход выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,молодежь,автомобиль,с детьми,доход ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,молодежь,недвижимость,без детей,доход выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,молодежь,образование,многодетный родитель,доход выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,зрелые,мероприятие,без детей,доход выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,молодежь,недвижимость,без детей,доход выше среднего
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,зрелые,недвижимость,без детей,доход выше среднего
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,зрелые,образование,без детей,доход ниже среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,молодежь,мероприятие,с детьми,доход ниже среднего
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,зрелые,недвижимость,без детей,доход выше среднего


Давайте посмотрим небольшую статистику по созданным нами категориям:

In [54]:
df1['category'].value_counts()

недвижимость    10810
автомобиль       4306
образование      4013
мероприятие      2323
Name: category, dtype: int64

In [55]:
df1['par_cat'].value_counts()

без детей               14089
с детьми                 6907
многодетный родитель      456
Name: par_cat, dtype: int64

In [56]:
df1['income_cat'].value_counts()

доход выше среднего    10726
доход ниже среднего    10726
Name: income_cat, dtype: int64

**Вывод**

Чаще всего люди обращаются за кредитом для операций с недвижимостью. Таких запросов больше, чем сумма всех запросов по другим категориям.

Чаще всего обращаются люди без детей. Скорее вего так происходит потому что у таких людей нет финансовых расходов на детей, и они могут позволить себе больше средств тратить на себя и больше рисковать.

Люди с доходом ниже среднего чуть чаще обращаются за кредитом, нежели люди с доходом выше среднего. Разница не велика, но мы видим что брать кредиты интересно людям с разными доходами.

"Коллективные" люди (состоящие в любом браке) намного чаще пользуются кредитом, вероятно это может быть связано с тем, что коллективно закрывать финаносвые вопросы легче путем распределения финансовой нагрузки.

Мы получили интересные данные благодаря категоризации данных

## Ответьте на вопросы

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим сводную таблицу по категориям родителей и выплатам.

In [57]:
children_pivot_table=df1.pivot_table(index='par_cat', columns = 'debt', values='gender', aggfunc= 'count')
children_pivot_table

debt,0,1
par_cat,,
без детей,13026,1063
многодетный родитель,417,39
с детьми,6268,639


Добавим столбец с суммой всех значений.

In [58]:
children_pivot_table['sum']=children_pivot_table[0]+children_pivot_table[1]
children_pivot_table

debt,0,1,sum
par_cat,,,
без детей,13026,1063,14089
многодетный родитель,417,39,456
с детьми,6268,639,6907


Теперь нам необходимо рассчитать доли по каждой категории и доли от всего объема данных. 
Для этого нам будет необходимо написать несколько функций:

In [59]:
def part_of_category(row):
    '''
    Функция вычисляет долю должников внутри категории:
    ======================================

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    result (float): заполняет ячейку значением
    '''

    result = (100*row[1])/row['sum']
    return result


print('Строка документации: \n', part_of_category.__doc__)

Строка документации: 
 
    Функция вычисляет долю должников внутри категории:

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    result (float): заполняет ячейку значением
    


Создадим столбец доли от категории "p_o_c", заполним его при помощи нашей функции.

In [60]:
children_pivot_table['p_o_c']=children_pivot_table.apply(part_of_category, axis=1)
children_pivot_table

debt,0,1,sum,p_o_c
par_cat,,,,
без детей,13026,1063,14089,7.544893
многодетный родитель,417,39,456,8.552632
с детьми,6268,639,6907,9.251484


Теперь создадим функцию для рассчета доли от всех данных:

In [61]:
def part_of_total(row):
    '''
    Функция вычисляет долю должников среди всех категорий:
    ======================================

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    total_result (float): заполняет ячейку значением
    '''

    total_result = (100*row[1])/len(df1)
    return total_result


print('Строка документации: \n', part_of_total.__doc__)

Строка документации: 
 
    Функция вычисляет долю должников среди всех категорий:

    Аргументы:
                    row (DataFrame): передаеся ряд датафрейма

    Возвращает:
                    total_result (float): заполняет ячейку значением
    


Создадим столбец "p_o_t" доли невозварата каждой категории от общей суммы данных, и заполним его при помощи функции:

In [62]:
children_pivot_table['p_o_t']=children_pivot_table.apply(part_of_total, axis=1)
children_pivot_table

debt,0,1,sum,p_o_c,p_o_t
par_cat,,,,,
без детей,13026,1063,14089,7.544893,4.955249
многодетный родитель,417,39,456,8.552632,0.181801
с детьми,6268,639,6907,9.251484,2.978743


**Вывод:**

В нашей выборке большинство обратившихся за кредитом - не имеют детей, доля должников среди них 7,53%. А доля от общего количества взявших кредит по имеющимся данным 4,95%. 

Группа многодетных родителей обратившихся за кредитом - численно самая маленькая. Доля должников внутри группы 8,55%. А доля от общего количества взявших кредит 0.18%

Доля должников среди людей "с детьми" 9,25%. А их доля от общего количества 2,97%.

Можно сделать вывод, что люди не имеющие детей реже становятся должниками. Процент от общего количества достаточно большой, но это обусловлено самой большой численностью группы.

Делаем вывод, что есть зависимость между наличием детей и возвратом кредита в срок.
Наличие ребенка негативно влияет на возврат кредита.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим сводную таблицу по семейному положению:

In [63]:
family_pivot_table=df1.pivot_table(index='family_status', columns = 'debt', values='gender', aggfunc= 'count')
family_pivot_table

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3762,388
женат / замужем,11407,931


Создадим столбец суммы:

In [64]:
family_pivot_table['sum']=family_pivot_table[0]+family_pivot_table[1]
family_pivot_table

debt,0,1,sum
family_status,,,
Не женат / не замужем,2536,274,2810
в разводе,1110,85,1195
вдовец / вдова,896,63,959
гражданский брак,3762,388,4150
женат / замужем,11407,931,12338


Применим наши функции рассчета долей к данной таблице и создадим столбцы "p_o_c" - доля внутри категории, "p_o_t" - доля от общего количества данных.

In [65]:
family_pivot_table['p_o_c']=family_pivot_table.apply(part_of_category, axis=1)
family_pivot_table['p_o_t']=family_pivot_table.apply(part_of_total, axis=1)
family_pivot_table

debt,0,1,sum,p_o_c,p_o_t
family_status,,,,,
Не женат / не замужем,2536,274,2810,9.750890,1.277270
в разводе,1110,85,1195,7.112971,0.396233
вдовец / вдова,896,63,959,6.569343,0.293679
гражданский брак,3762,388,4150,9.349398,1.808689
женат / замужем,11407,931,12338,7.545793,4.339922


**Вывод**

Хуже всего платят кредит люди не состоящие в браке или живущие гражданским браком 9,75% и 9,32% соответсвтенно. Лучше всего платят люди потерявшие супруга. Самая большая категория людей состоящие в браке. Этим обусловлен большой процент от общей выборки.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу по уровню дохода:

In [66]:
income_pivot_table=df1.pivot_table(index='income_cat', columns = 'debt', values='gender', aggfunc= 'count')
income_pivot_table

debt,0,1
income_cat,,
доход выше среднего,9897,829
доход ниже среднего,9814,912


Добавим столбец с суммой:

In [67]:
income_pivot_table['sum']=income_pivot_table[0]+income_pivot_table[1]
income_pivot_table

debt,0,1,sum
income_cat,,,
доход выше среднего,9897,829,10726
доход ниже среднего,9814,912,10726


Применим наши функции рассчета долей к данной таблице и создадим столбцы "p_o_c" - доля внутри категории, "p_o_t" - доля от общего количества данных.

In [68]:
income_pivot_table['p_o_c']=income_pivot_table.apply(part_of_category, axis=1)
income_pivot_table['p_o_t']=income_pivot_table.apply(part_of_total, axis=1)
income_pivot_table

debt,0,1,sum,p_o_c,p_o_t
income_cat,,,,,
доход выше среднего,9897,829,10726,7.728883,3.864442
доход ниже среднего,9814,912,10726,8.502704,4.251352


**Вывод**

Чаще всего становятся должниками банков люди с суммарным доходом ниже среднего, однако разница не велика, всего в 0,5%. Большие заработки - не гарантируют своевременной оплаты кредитов.

### - Как разные цели кредита влияют на его возврат в срок?

Создадим сводную таблицу по назначениям кредита:

In [69]:
category_pivot_table=df1.pivot_table(index='category', columns = 'debt', values='gender', aggfunc= 'count')
category_pivot_table

debt,0,1
category,,
автомобиль,3903,403
мероприятие,2137,186
недвижимость,10028,782
образование,3643,370


Добавим столбец с суммой:

In [70]:
category_pivot_table['sum']=category_pivot_table[0]+category_pivot_table[1]
category_pivot_table

debt,0,1,sum
category,,,
автомобиль,3903,403,4306
мероприятие,2137,186,2323
недвижимость,10028,782,10810
образование,3643,370,4013


Применим наши функции рассчета долей внутри категории и от общей массы. Заполним данными функциями столбцы  "p_o_c" и  "p_o_t": 

In [71]:
category_pivot_table['p_o_c']=category_pivot_table.apply(part_of_category, axis=1)
category_pivot_table['p_o_t']=category_pivot_table.apply(part_of_total, axis=1)
category_pivot_table

debt,0,1,sum,p_o_c,p_o_t
category,,,,,
автомобиль,3903,403,4306,9.359034,1.878613
мероприятие,2137,186,2323,8.006888,0.867052
недвижимость,10028,782,10810,7.234043,3.645348
образование,3643,370,4013,9.220035,1.724781


**Вывод**

Люди взявшие кредит на образование или автомобиль - чаще становятся должниками чем люди взявшие кредит на недвижимость, свадьбу или ремонт.

## Общий вывод

На основании выводов из пункта 3, мы можем выделить группу повышенного риска, а так же группу наименьшего риска.

Самая рисковая групппа людей:

- люди с детьми, не состоящие в браке, имеющие доход ниже среднего, берущие кредит для образования или покупки автомобиля, являются самой высокорисковой группой людей.

Самая надежная группа людей:

- люди без детей, вдовцы, а так же сотоящие в браке или разведеённые, имеющие доход выше среднего, берущие кредит для операций с недвижимостью, являются самой низкорисковой группой людей. 

Самая надежная в плане выплат кредитов является группа люедй не имеющих детей - по статистике 7,5% людей имеют задолженности по кредитам.
Самая безответственная группа людей - люди имеющие 1-2 ребёнка.9,25% людей данной группы имею задолженности перед банокм. Многодетные родители имеют статистику чуть лучше и показали 8,55%.

Самые надежные группы людей по семейному положеню вдовы, люди состоящие в браке или в разводе, по статистике должники составляют 6.56%, 7.54%, 7.11% соответственно. 
Самые безответственные группы людей - люди состоящие в гражданском браке или не состоящие в браке вообще, таких людей 9.32% и 9.75% соответственно.

Люди имеющие уровень дохода выше среднего являются более надежными плательщиками их 7,83%
Люди имеющие уровень дохода ниже среднего чаще становятся должниками примерно в 8,38% случаев. Хотя разника между этими группами не велика.

Самы надежные клиенты - люди берущие займ для операций с недвижимостью, таких должников 7.23%.
Саой безответственной группой людей - являются люди берущией займ на покупку автомобиля или для получения образования. Таких должников 9.35% и 9.21% соответственно.